# Sign Language Detection Project



In [108]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
# need to launch VS Code as Administrator to use mediapipe?
from tensorflow.keras.utils import to_categorical

In [109]:
# %pip install tensorflow-gpu

In [115]:
# Path for exported data, numpy arrays

# Actions that we try to detect
# actions = np.array(['yes', 'no', 'PeaceAmongWorlds'])
actions = np.array(['Hi', 'Yes', 'No', 'ThankYou', 'ILoveYou'])

# Thirty videos worth of data
no_sequences = 10

# Videos are going to be 30 frames in length
sequence_length = 30

DATA_PATH = os.path.join('MP_Data')

# 6. Preprocess Data and Create Labels and Features

In [111]:
label_map = {label:num for num, label in enumerate(actions)}

In [112]:
label_map

{'Hi': 0, 'Yes': 1, 'No': 2, 'ThankYou': 3, 'ILoveYou': 4}

In [116]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [117]:
np.array(sequences).shape

(50, 30, 1662)

In [118]:
np.array(labels).shape

(50,)

In [119]:
X = np.array(sequences)

In [120]:
y = to_categorical(labels).astype(int)

In [121]:
np.save('X', X)
np.save('y', y)

In [122]:
# not being used now - mounting google drive to be able to access files in the folder
# from google.colab import drive
# drive.mount('/content/drive/MyDrive')

In [123]:
# the paths need to be updated accordingly
X = np.load('X.npy')
y = np.load('y.npy')

In [124]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [125]:
y_test.shape

(3, 5)

# 7. Build and Train LSTM Neural Network

In [126]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [127]:
import os

In [128]:
log_dir = os.path.join('Logs')


In [129]:
tb_callback = TensorBoard(log_dir=log_dir)

In [130]:
actions.shape[0]

5

In [131]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))
# relu = Rectified Linear Activation Function (returns 0 if it receives any negative input, but for any positive value x it returns that value back)

c:\Users\Cr4sZ\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [132]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [133]:
"""
tb_callback = TensorBoard(log_dir=log_dir). log_dir is the path of the directory where to save the log files to be parsed by TensorBoard.
TensorBoard is a visualization tool provided with TensorFlow. This callback logs events for TensorBoard, including:
Metrics summary plots, Training graph visualization, Activation histograms, Sampled profiling
"""
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000


2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - categorical_accuracy: 0.2289 - loss: 2.1944
Epoch 2/2000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - categorical_accuracy: 0.2006 - loss: 2.4262
Epoch 3/2000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - categorical_accuracy: 0.1693 - loss: 23.8423
Epoch 4/2000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - categorical_accuracy: 0.2006 - loss: 6.8579
Epoch 5/2000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - categorical_accuracy: 0.2431 - loss: 7.3813
Epoch 6/2000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - categorical_accuracy: 0.1902 - loss: 5.5569
Epoch 7/2000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - categorical_accuracy: 0.2081 - loss: 5.2200
Epoch 8/2000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - categorical_accuracy: 0.2886 - loss: 5.8932
Epoch 9/2000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - categorical_accuracy: 0.1864 - loss: 7.5595
Epoch 10/2000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - categorical_accuracy: 0.2327 - loss: 2.6647
Epoch 11/2000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s

In [134]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_12 (LSTM)                  │ (None, 30, 64)         │       442,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_13 (LSTM)                  │ (None, 30, 128)        │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_14 (LSTM)                  │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,790,225 (6.83 MB)

 Trainable params: 596,741 (2.28 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,193,484 (4.55 MB)

## 8. Make predictions

In [135]:
res = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step


In [136]:
res.shape

(3, 5)

In [137]:
res

array([[9.3829179e-01, 9.6604999e-05, 6.1611567e-02, 9.6247552e-09,
        2.3883209e-10],
       [9.9970657e-01, 1.4414839e-08, 2.9343073e-04, 3.8046992e-12,
        7.5493160e-13],
       [2.7145086e-07, 8.3025950e-01, 1.2047979e-03, 1.3797027e-01,
        3.0565217e-02]], dtype=float32)

In [138]:
actions[np.argmax(res[0])]

'Hi'

In [139]:
len(y_test)
# y_test[1]

3

In [140]:
actions[np.argmax(y_test[len(y_test)-1])]

'No'

## 9. Save weights

In [141]:
model.save("weights_with_facemesh.keras")

## 10. Evaluation using confusion matrix and accuracy

In [142]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [143]:
yhat = model.predict(X_train)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step  


In [144]:
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [145]:
multilabel_confusion_matrix(ytrue, yhat)
# Confusion Matrix = True Positive, True Negative, False Positive, False Negative

array([[[38,  1],
        [ 0,  8]],

       [[37,  0],
        [ 2,  8]],

       [[36,  2],
        [ 1,  8]],

       [[36,  1],
        [ 2,  8]],

       [[35,  2],
        [ 1,  9]]], dtype=int64)

In [146]:
accuracy_score(ytrue, yhat)

0.8723404255319149

# ToDo  
1. Improve Accuracy
    - train for more epochs (often: more accuracy, but might lead to overfitting)
    - fine-tune hyperparameters